In [13]:
import json
import spacy
import os
rule_tag_dict_jj = {}
rule_without_pos = {}
#to store tags rule wise.
nlp = spacy.load("en_core_web_sm")
output_dir = 'my_mixed_random/'
nlp2 = spacy.load(output_dir)
ent_list = []
ent_without_pos = []
file_array = os.listdir('data_rules/data')
for file_name in file_array:
    if file_name.endswith(".json"):
        f = open('data_rules/data/'+file_name)
        object_json2 = json.load(f)
        jj = list({v['id']:v for v in object_json2}.values())
        for rule in jj:
            if ('w.e.f.' not in rule['rule']) and ('_' not in rule['rule']):
            #     mod_text = add_tags(rule['text'],rule['tags']['tag_name'], rule['rule'])
            #     print((rule['rule'][rule['span']['start']:rule['span']['end']], rule['text']))
                ent_list.append({'label':rule['tags']['tag_name'], 'start':rule['span']['start'], 'end':rule['span']['end'], 'text':rule['text']})
                ent_without_pos.append({'label':rule['tags']['tag_name'], 'start':rule['span']['start'], 'end':rule['span']['end'], 'text':rule['text']})
            #     mod_text = adding_xml_tags(rule['text'],rule['tags']['tag_name'], '<b>'+rule['rule']+'</b' )
                doc2 = nlp2(rule['rule'])
                for ent in doc2.ents:
            #         print(mod_text)
            #         mod_text = add_tags(ent.text,  ent.label_, mod_text)
                    ent_list.append({'label':ent.label_, 'start':ent.start, 'end':ent.end, 'text':ent.text})
                    ent_without_pos.append({'label':ent.label_, 'start':ent.start, 'end':ent.end, 'text':ent.text})

                doc = nlp(rule['rule'])
            #     print(rule['rule'])
                for token in doc:
                    if token.pos_ not in ["PUNCT"]:
            #             print(mod_text)
            #             mod_text = add_tags(token.text,  token.pos_, mod_text)
            #             ent_list.append({'tag_name':token.pos_, 'start':token.start, 'end':token.end, 'text':token.text})
            #             pass
                        ent_list.append({'label':token.pos_, 'start':token.idx, 'end': token.idx+len(token.text), 'text':token.text})
                #sorting spans.
            #     ent_list = sorted(ent_list,
            #                           key=lambda k: (k['start']))
            rule_tag_dict_jj[rule['rule']] = ent_list
            rule_without_pos[rule['rule']] = ent_without_pos
            ent_list = []
            ent_without_pos = []
    #     break
# rule_tag_dict_jj

In [2]:
import pickle
#Function for pickling variables.
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
save_object(rule_tag_dict_jj,'rule_tags_all_regulations_with_pos.pkl')
save_object(rule_without_pos, 'rule_tags_all_regulations_without_pos.pkl')

In [14]:
def ordered_sublist(l1, l2):
    length = len(l1)
    for i in range(len(l2) - length + 1):
        if all(l1[j] == l2[j + i] for j in range(length)):
            return True
    return False

def get_key(val, my_dict): 
    for key, value in my_dict.items(): 
         if val == value: 
             return key 
  
    return "key doesn't exist"
#this is only for a single rule.(hopefully)
#returns start and end index of rule text.
def get_recurred_phrase_tags(list_tag_dicts, phrase_list):
    k = phrase_list[0]
    for d in range(len(list_tag_dicts)):
        specific_list = []
        if list_tag_dicts[d]['label'] == k:
#             print(list_tag_dicts[d:])
#             print(phrase_list)
            t = list_tag_dicts[d:d+len(phrase_list)]
            probable_sublist = [d['label'] for d in t]
#             print((len(probable_sublist), len(phrase_list)))
            if phrase_list==probable_sublist:
                specific_list = t
                return specific_list
                
    return specific_list            
#     if specific_list:
#         return specific_list
#         make_dict = {'text':s[specific_list[0]['start']: specific_list[-1]['end']]}
#         #sorting list_ents.
# #         ent_list = sorted(ent_list,
# #                               key=lambda k: (k['start']))
#         make_dict['ents'] = specific_list
# #         make_dict["title"]= title
#         ex = [make_dict]
#         return ex
#     else:
#         return False
                
        
def tagged_returns(s, specific_list):
    make_dict = {'text':s}
    make_dict['ents'] = specific_list
#     make_dict["title"]= title
    ex = [make_dict]
    return ex

def tag_words(s):
    nlp = spacy.load("en_core_web_sm")
    output_dir = 'my_mixed_random/'
    nlp2 = spacy.load(output_dir)
    ent_list = []
    doc2 = nlp2(s)
    for ent in doc2.ents:
        ent_list.append({'label':ent.label_, 'start':ent.start, 'end':ent.end})
    doc = nlp(s)
    for token in doc:
        ent_list.append({'label':token.pos_, 'start':token.idx, 'end': token.idx+len(token.text)})
    ent_list = sorted(ent_list,
                          key=lambda k: (k['start']))
    sign_list = []
    for tag in ent_list:
        sign_list.append(tag['label'])
    return sign_list

In [29]:
collecting_common  = {}
for k in rule_tag_dict_jj:
    d = {}
    f = {}
    for m in rule_tag_dict_jj[k]:
        try:
#             print(m['start'])
#             print(type(m['start']))
            u = (m['start'])
            if u not in d.keys():
                d[u] = [m['label']]
            else:
                d[u].append(m['label'])
        except:
            pass
    for key in d:
        if len(d[key])>1:
            f[key] = ' '.join(d[key])
        else:
            f[key] = d[key][0]
        
#     collecting_common.append((k,f))
    collecting_common[k] = f
#     new['rule'] = k
#     break
collecting_common

{'SECURITIES AND EXCHANGE BOARD OF INDIA': {33: 'coref PROPN',
  0: 'Person In charge PROPN',
  11: 'CCONJ',
  15: 'PROPN',
  24: 'PROPN',
  30: 'ADP'},
 'SECURITIES AND EXCHANGE BOARD OF INDIA (RESEARCH ANALYSTS)': {33: 'coref PROPN',
  0: 'Person In charge PROPN',
  11: 'CCONJ',
  15: 'PROPN',
  24: 'PROPN',
  30: 'ADP',
  40: 'PROPN',
  49: 'PROPN'},
 'No. LAD-NRO/GN/2014-15/07/1414.- In exercise of the powers conferred by sub-section (1) of section 30 read with clause (b) of sub-section (2) of section 11 of the Securities and Exchange Board of India Act, 1992(15 of 1992), the Securities and Exchange Board of India hereby, makes the following regulations, to put in place a framework to register and regulate research analysts, namely;-': {225: 'coref DET',
  4: 'Company PROPN',
  16: 'Legal Doc',
  32: 'Legal Doc',
  42: 'Person In charge',
  56: 'Person In charge',
  68: 'Legal Doc',
  75: 'Object',
  0: 'INTJ',
  8: 'NOUN',
  11: 'SYM',
  12: 'NOUN',
  20: 'VERB',
  33: 'ADP',
  36

In [30]:
tag_collection_by_rules = []
rule_pattern = {}
for rule in collecting_common:
    sorted_dict = dict(sorted(collecting_common[rule].items()))
    l = list(sorted_dict.values())
#     flat_list = [item for sublist in l for item in sublist]
    tag_collection_by_rules.append(l)
#     print(rule[0])
    rule_pattern[rule] = l
# tag_collection_by_rules
rule_pattern

{'SECURITIES AND EXCHANGE BOARD OF INDIA': ['Person In charge PROPN',
  'CCONJ',
  'PROPN',
  'PROPN',
  'ADP',
  'coref PROPN'],
 'SECURITIES AND EXCHANGE BOARD OF INDIA (RESEARCH ANALYSTS)': ['Person In charge PROPN',
  'CCONJ',
  'PROPN',
  'PROPN',
  'ADP',
  'coref PROPN',
  'PROPN',
  'PROPN'],
 'No. LAD-NRO/GN/2014-15/07/1414.- In exercise of the powers conferred by sub-section (1) of section 30 read with clause (b) of sub-section (2) of section 11 of the Securities and Exchange Board of India Act, 1992(15 of 1992), the Securities and Exchange Board of India hereby, makes the following regulations, to put in place a framework to register and regulate research analysts, namely;-': ['INTJ',
  'Company PROPN',
  'NOUN',
  'SYM',
  'NOUN',
  'Legal Doc',
  'VERB',
  'Legal Doc',
  'ADP',
  'NOUN',
  'Person In charge',
  'ADP',
  'DET',
  'NOUN',
  'Person In charge',
  'VERB',
  'Legal Doc',
  'ADP',
  'NOUN',
  'Object',
  'NOUN',
  'NUM',
  'ADP',
  'NOUN',
  'NUM',
  'VERB',
  '

In [33]:
from collections import Counter
def get_seq(x, M):
    index_for_length_m = M - 1
    for v in [l for l in x if len(l) >= M]:
        for i in [i for i, v in enumerate(v[index_for_length_m:], start=index_for_length_m)]:
            # convert to str to be hashable
            yield str(v[i - index_for_length_m : i + 1])

def process_chunk(x, M, N):
    return Counter(get_seq(x, M)).most_common(N)

from itertools import combinations 
  
def rSubset(arr, r): 
  
    # return list of all subsets of length r 
    # to deal with duplicate subsets use  
    # set(list(combinations(arr, r))) 
    return list(combinations(arr, r)) 


In [54]:
import nltk
def similarity_between_rules(r):
    rule1 = r[0]
    rule2 = r[1]
    s1 = tag_words(rule1)
    s2 = tag_words(rule2)
    return nltk.edit_distance(s1, s2)

r1= '(i)any person who is or has during the six months prior to the concerned act been associated with a company, directly or indirectly, in any capacity including by'
r2 = 'reason of frequent communication with its officers or by being in any contractual, fiduciary or employment relationship or by being a director, officer or an employee of the company or holds any position including a professional or'
similarity_between_rules((r1,r2))

33

In [58]:
import ast
from spacy import displacy
w = process_chunk(tag_collection_by_rules, M=20, N=10)
for k in w:
    l1 = ast.literal_eval(k[0])
#     print(l1)
    rule_list = []
    #retrieving all rules matching with this tag sequence.
    for rule_sign in rule_pattern.values():
        if ordered_sublist(l1,rule_sign):
            rule = get_key(rule_sign, rule_pattern)
            rule_list.append(rule)
            #print rule first for the sake of reference.
            #Now printing exact recurring tagged phrase in the rule for detailed view.
            specific_list = get_recurred_phrase_tags(rule_tag_dict_jj[rule], l1)
            ex = tagged_returns(rule, specific_list)
#             html = displacy.render(ex, style="ent", manual=True, page=True)
#         print(rule_list)
    rad = rSubset(rule_list, 2)
    for r in rad:
        print(r[0])
        print(r[1])
        print(similarity_between_rules(r))
#         print(rad)
    print("End of rules for the pattern.")
            
    

End of rules for the pattern.


End of rules for the pattern.


End of rules for the pattern.


End of rules for the pattern.


End of rules for the pattern.


End of rules for the pattern.


End of rules for the pattern.


End of rules for the pattern.


End of rules for the pattern.


End of rules for the pattern.


In [61]:
rule_list = list(set(rule_list))
rad = rSubset(rule_list, 2)
for r in rad:
    print(r[0])
    print(r[1])
    print(similarity_between_rules(r))
    print('\n')

27A. (1) The Board may, exempt any person or class of persons from the operation of all or any of the provisions of these regulations for a period as may be specified but not exceeding twelve months, for furthering innovation in technological aspects relating to testing new products, processes, services, business models, etc. in live environment of regulatory sandbox in the securities markets.
93A. (1) The Board may, exempt any person or class of persons from the operation of all or any of the provisions of these regulations for a period as may be specified but not exceeding twelve months, for furthering innovation in technological aspects relating to testing new products, processes, services, business models, etc. in live environment of regulatory sandbox in the securities markets.
1


27A. (1) The Board may, exempt any person or class of persons from the operation of all or any of the provisions of these regulations for a period as may be specified but not exceeding twelve months, fo

6


but not exceeding twelve months, for furthering innovation in technological aspects relating to testing new products, processes, services, business models, etc. in live environment of regulatory sandbox in the securities markets.
26.(1) The Board may, exempt any person or class of persons from the operation of all or any of the provisions of these regulations for a period as may be specified but not exceeding twelve months, for furthering innovation in technological aspects relating to testing new products, processes, services, business models, etc. in live environment of regulatory sandbox in the securities markets.
42


but not exceeding twelve months, for furthering innovation in technological aspects relating to testing new products, processes, services, business models, etc. in live environment of regulatory sandbox in the securities markets.
of the provisions of these regulations for a period as may be specified but not exceeding twelve months, for furthering innovation in te

In [ ]:
rulewise_files = {}
file_array = os.listdir('data_rules/data')
for file_name in file_array:
    if file_name.endswith(".json"):
        f = open('data_rules/data/'+file_name)
        object_json2 = json.load(f)
        jj = list({v['id']:v for v in object_json2}.values())
        local_rules = []
        for rule in jj:
            if ('w.e.f.' not in rule['rule']) and ('_' not in rule['rule']):
                local_rules.append(rule['rule'])
        rulewise_files[file_name] = local_rules

In [7]:
collecting_common

{'SECURITIES AND EXCHANGE BOARD OF INDIA': {33: 'coref PROPN',
  0: 'Person In charge PROPN',
  11: 'CCONJ',
  15: 'PROPN',
  24: 'PROPN',
  30: 'ADP'},
 'SECURITIES AND EXCHANGE BOARD OF INDIA (PROHIBITION OF INSIDER TRADING) REGULATIONS, 2015': {33: 'coref PROPN',
  0: 'Person In charge PROPN',
  11: 'CCONJ',
  15: 'PROPN',
  24: 'PROPN',
  30: 'ADP',
  40: 'NOUN',
  52: 'ADP',
  55: 'PROPN',
  63: 'NOUN',
  72: 'NOUN',
  85: 'NUM'},
 'No. LAD-NRO/GN/2014-15/21/85.- In exercise of the powers conferred by section 30 read with clause (g) of sub-section (2) of section 11 and clause (d) and clause (e) of section 12A of the Securities and Exchange Board of India Act, 1992 (15 of 1992), the Board hereby makes the following regulations, to put in place a framework for prohibition of insider trading in securities and to strengthen the legal framework thereof, namely:—': {220: 'coref PROPN',
  4: 'Company PROPN',
  20: 'Legal Doc NUM',
  25: 'Legal Doc',
  35: 'Legal Doc',
  40: 'Legal Doc',


In [30]:
process_chunk(tag_collection_by_rules, M=100, N=10)

[("['DET', 'Object', 'Legal Doc', 'ADJ', 'Object', 'NOUN', 'Object', 'Object', 'Penalty', 'Penalty', 'ADP', 'DET', 'Legal Doc', 'ADJ', 'Penalty', 'Penalty', 'Penalty', 'NOUN', 'Object', 'Legal Doc', 'Investigation CCONJ', 'Investigation', 'PRON', 'Investigation', 'Object', 'ADV', 'Object', 'Penalty', 'Penalty VERB', 'Penalty', 'Penalty', 'CCONJ', 'Penalty', 'VERB', 'Legal Doc', 'NOUN', 'DET', 'NOUN', 'NOUN', 'NOUN', 'CCONJ', 'NOUN', 'VERB', 'VERB', 'CCONJ', 'VERB', 'ADP', 'coref DET', 'ADJ', 'NOUN', 'DET', 'NOUN', 'NOUN', 'CCONJ', 'NOUN', 'VERB', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', 'VERB', 'ADP', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'DET', 'NOUN', 'ADJ', 'NOUN', 'CCONJ', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'CCONJ', 'NOUN', 'SCONJ', 'VERB', 'VERB', 'VERB', 'ADJ', 'SCONJ', 'SCONJ', 'DET', 'ADJ', 'NOUN', 'AUX', 'ADV', 'AUX', 'VERB']",
  2),
 ("['NUM', 'Person - General', 'Person - General', 'NOUN', 'Assets CCONJ', 'Company', 'NOUN', 'Objec

In [9]:
tag_collection_by_rules

[['Person In charge PROPN', 'CCONJ', 'PROPN', 'PROPN', 'ADP', 'coref PROPN'],
 ['Person In charge PROPN',
  'CCONJ',
  'PROPN',
  'PROPN',
  'ADP',
  'coref PROPN',
  'NOUN',
  'ADP',
  'PROPN',
  'NOUN',
  'NOUN',
  'NUM'],
 ['INTJ',
  'Company PROPN',
  'NOUN',
  'SYM',
  'NOUN',
  'Legal Doc NUM',
  'Legal Doc',
  'ADP',
  'NOUN',
  'Legal Doc',
  'Legal Doc',
  'ADP',
  'DET',
  'Person In charge',
  'NOUN',
  'VERB',
  'Person In charge',
  'ADP',
  'Legal Doc NOUN',
  'Object',
  'NUM',
  'Penalty',
  'Penalty',
  'VERB',
  'ADP',
  'NOUN',
  'NOUN',
  'ADP',
  'NOUN',
  'NOUN',
  'NUM',
  'ADP',
  'NOUN',
  'NUM',
  'CCONJ',
  'NOUN',
  'NOUN',
  'CCONJ',
  'NOUN',
  'PROPN',
  'ADP',
  'NOUN',
  'NUM',
  'ADP',
  'DET',
  'PROPN',
  'CCONJ',
  'PROPN',
  'PROPN',
  'ADP',
  'coref PROPN',
  'PROPN',
  'NUM',
  'NUM',
  'ADP',
  'NUM',
  'DET',
  'PROPN',
  'NOUN',
  'VERB',
  'DET',
  'VERB',
  'NOUN',
  'PART',
  'VERB',
  'ADP',
  'NOUN',
  'DET',
  'NOUN',
  'ADP',
  'NOUN',

In [2]:
import json
import os
import nltk
import spacy
from operator import itemgetter
import itertools
import random
import Levenshtein
import difflib
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
def depickle(file):
    file = open(file, 'rb')
    data = pickle.load(file)
    file.close()
    return data
def load_json_file(file):
    with open(file) as f:
        data = json.load(f)
    return data
all_rules = load_json_file('ujwal_annotated_all_json.json')
rule_tag_dict = {}
rule_without_pos = {}
#to store tags rule wise.
nlp = spacy.load("en_core_web_sm")
output_dir = 'best_trained/'
nlp2 = spacy.load(output_dir)
count = 0
for rule in all_rules.values():
    ent_list = []
    count += 1
    for labs,text in zip(rule['ent_labels'],rule['ents']):
        ent_list.append({'start':rule['text'].find(text), 'end':rule['text'].find(text)+len(text), 'label':labs})
    doc2 = nlp2(rule['text'])
    for ent in doc2.ents:
        ent_list.append({'label':ent.label_, 'start':ent.start_char, 'end':ent.end_char})
    doc = nlp(rule['text'])
    for token in doc:
        if token.pos_  in ["VERB", "ADJ"]:
            ent_list.append({'label':token.pos_, 'start':token.idx, 'end': token.idx+len(token.text)})
    #sorting spans.
    ent_list = sorted(ent_list,
                          key=lambda k: (k['start']))
    rule_tag_dict[rule['text']] = ent_list
rule_tag_dict
count

6012

In [3]:
save_object(rule_tag_dict,'all_rules_all_annotations.pkl')